In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Generating Data

In [2]:
def generate_date(year_1 = 1980, year_2 = 2000):
    return datetime.datetime(year_1, 1, 1) + datetime.timedelta(days= np.random.randint((datetime.datetime(year_2, 12,31) - datetime.datetime(year_1, 1,1)).days))  

In [3]:
emp_counts = int(1e3)
# Creating DataFrame
data = pd.DataFrame(data = {'id' : np.random.randint(0, emp_counts*10, emp_counts), 
                            'gender' : ['Male' if np.random.rand()<0.5 else 'Female' for i in range(emp_counts)],
                            'dob' : [generate_date() for i in range(emp_counts)],
                            'doh' : [generate_date(year_1 = 2008, year_2 = 2023) for i in range(emp_counts)],
                            'salary' : 6e6+ np.random.uniform(low = -3e6, high = 10e6, size = emp_counts) },)
data['dob'] = pd.to_datetime(data['dob'])
data['doh'] = pd.to_datetime(data['doh'])
data

,id,gender,dob,doh,salary
0,9339,Male,1985-05-30,2015-09-11,7.558607e+06
1,9221,Male,1990-10-01,2014-05-26,1.219255e+07
2,5081,Female,1985-11-16,2015-04-28,1.317599e+07
3,6692,Female,1988-11-17,2009-11-18,1.554695e+07
4,4973,Male,1999-06-07,2014-08-05,4.734278e+06
...,...,...,...,...,...
995,4564,Female,1997-10-17,2021-07-28,3.259245e+06
996,3226,Male,1980-05-22,2023-08-22,8.135059e+06
997,916,Male,1998-01-27,2018-04-24,3.294824e+06
998,6703,Male,1985-07-23,2013-04-21,3.589379e+06


In [4]:
val_date = pd.Timestamp('2023-12-31')
data['age'] = np.round((val_date- data.dob)/np.timedelta64(1, 'Y'),2)
data['yos'] = np.round((val_date- data.doh)/np.timedelta64(1, 'Y'),2)

# Actuarial Assumptions

# Demographic Assumptions

Assume death probability follows 4th Indonesia Mortality Table. Disability probability is 1% of the former mortality table and resignation rate is 1% decreasing linearly from age 22 to age 56 (pension age).

In [5]:
mortality_base = pd.read_csv(r'C:\Users\leona\Documents\GitHub\post-employment-benefit\data\TMI IV.csv')
mortality_base
pension_age = 56

In [6]:
def resignation_rate(entry_age, start_age = 22, end_age = pension_age, start_rate = 0.01, end_rate = 0):
    return start_rate +(end_rate - start_rate)*(np.arange(entry_age, end_age) - start_age)/(end_age - start_age) 

In [7]:
def demographic_table(table = mortality_base, employee_gender = None, employee_age = None, pension_age = pension_age):
    death = mortality_base[employee_gender].loc[np.floor(employee_age):pension_age-1]
    disable = death*0.01
    resign = resignation_rate(entry_age = employee_age)
    return pd.DataFrame(data = {'death': death, 'disable': disable.values, 'resign' : resign}, 
                        index = np.arange(np.floor(employee_age), pension_age))
    

In [8]:
demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]).shape

(18, 3)

In [9]:
a = np.ones((10, 3))
a[11] = [1,1,1]

IndexError: index 11 is out of bounds for axis 0 with size 10

In [ ]:
def service_table(demographic_tbl):
    survive = np.ones((demographic_tbl.shape[0], 4))
    for i in range(demographic_tbl.shape[0]):
        survive[i,1] = survive[i,0] * demographic_tbl.iloc[i,0] * (1 - demographic_tbl.iloc[i,1]) * (1 - demographic_tbl.iloc[i,2])
        survive[i,2] = survive[i,0] * demographic_tbl.iloc[i,1] * (1 - demographic_tbl.iloc[i,0]) * (1 - demographic_tbl.iloc[i,2])
        survive[i,3] = survive[i,0] * demographic_tbl.iloc[i,2] * (1 - demographic_tbl.iloc[i,0]) * (1 - demographic_tbl.iloc[i,1])
        try : 
            survive[i+1,0] = survive[i,0] - np.sum(survive[i,1:])  
        except : 
            survive = np.append(survive, np.array([[survive[i,0] - np.sum(survive[i,1:]), 0, 0, 0]]), axis = 0)
    return pd.DataFrame(data = survive, columns = ['survive', 'death', 'disable', 'resign'], index = np.append(demographic_tbl.index, 56))

In [ ]:
service_table(demographic_tbl = demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]))

,survive,death,disable,resign
25.0,1.000000,0.000377,0.000004,0.009014
26.0,0.990605,0.000412,0.000004,0.008638
27.0,0.981551,0.000448,0.000004,0.008270
28.0,0.972829,0.000473,0.000005,0.007910
29.0,0.964441,0.000498,0.000005,0.007558
30.0,0.956380,0.000532,0.000005,0.007214
31.0,0.948629,0.000565,0.000006,0.006876
32.0,0.941183,0.000598,0.000006,0.006545
33.0,0.934033,0.000640,0.000006,0.006221
34.0,0.927166,0.000682,0.000007,0.005902


## Financial Assumption

In [ ]:
sev_svc = pd.DataFrame({'severance': [min(i+1,9) for i in range(60)],
                        'service' : [0,0,0,2,2,2,3,3,3,
                                     4,4,4,5,5,5,6,6,6,
                                     7,7,7,8,8,8,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]})
ben_fac = pd.DataFrame({'retire': 1.75*sev_svc['severance']+sev_svc['service'],
                        'death': 2*sev_svc['severance']+sev_svc['service'],
                        'disable': 2*sev_svc['severance']+sev_svc['service'],
                        'resign': [1]*sev_svc.shape[0]})
ben_fac

In [16]:
yield_curve  = pd.read_csv(r'C:\Users\leona\Documents\GitHub\post-employment-benefit\data\YieldCurve1Sep.csv')
yield_curve

,enor Year,Today
0,0.1,0.061580
1,1.0,0.061591
2,2.0,0.061629
3,3.0,0.061763
4,4.0,0.062007
5,5.0,0.062345
6,6.0,0.062754
7,7.0,0.063204
8,8.0,0.063671
9,9.0,0.064134


In [45]:
def spot_rates(yield_curve):
    spot_rate = np.zeros(yield_curve.shape[0])
    t = yield_curve.iloc[:,0]
    spot_rate[0] = yield_curve.iloc[0,1]
    spot_rate[1] = yield_curve.iloc[1,1]
    for i in range(2,yield_curve.shape[0]):
        sum = 0
        for j in range(1,i):
            sum += yield_curve.iloc[j,1]/(1+spot_rate[j])**t[j]
        spot_rate[i] = ((1+yield_curve.iloc[i,1])/(1-sum))**(1/t[i]) - 1   
    return pd.DataFrame(data = {'spot_rate' : spot_rate})

In [46]:
spot_rates(yield_curve)

,spot_rate
0,0.061580
1,0.061591
2,0.061610
3,0.061661
4,0.061747
5,0.061867
6,0.062015
7,0.062185
8,0.062370
9,0.062566
